In [ ]:
import os
import shutil
import glob
import subprocess
import pandas as pd
import geopandas as gpd
import dask
from dask.distributed import Client, progress, LocalCluster
from pyFIRS.wrappers import lastools, fusion
from pyFIRS.utils import clean_dir, clean_buffer_polys, clip_tile_from_shp, convert_project, PipelineError, fname

### Setting up parallel computing using `dask.distributed`
`LAStools` offers native multi-core processing as an optional argument (`cores`) supplied to its command-line tools. `FUSION` command line tools do not. To enable parallel processing of `FUSION` commands, we'll use `dask.distributed` to schedule the processing of tiles in asynchronous parallel batches. This approach also offers us the ability to track progress using a progress bar.

You'll first need to launch a parallel computing cluster. 

In [ ]:
cluster=LocalCluster(scheduler_port=7001, diagnostics_port=7002)
c = Client(cluster)
num_cores = len(c.ncores()) # identify how many workers we have

At this point, you should also be able to view an interactive dashboard on port 7002. If you're executing this on a remote server, you'll need to set up port forward so you can view the dashboard on your local machine's browser. Once you've done that, or if you're processing on your own machine, you can view the dashboard at [http://localhost:7002/status](http://localhost:7002/status).

# Let's get to work with some lidar data

We'll define where we can find the binary executables for LAStools and FUSION command line tools.

In [ ]:
las = lastools.useLAStools('/storage/lidar/LAStools/bin')
fus = fusion.useFUSION('/storage/lidar/FUSION/')

We'll create working directories for raw (imported with modest clean-up from source files), interim, and processed data.

In [ ]:
# where the imported lidar data is currently stored
workdir = os.path.abspath('/storage/lidar/olc_metro_2014/')
# define data handling directories
interim = os.path.join(workdir,'interim')
processed = os.path.join(workdir,'processed')
layers = os.path.join(interim, 'layers')


# the coordinate reference system we'll be working with
target_epsg = 26910 # utm 10 N

In [ ]:
def log_error(tile_id, process, error_msg):
    logfile = os.path.join(interim, 'failed', tile_id + '.txt')
    os.makedirs(os.path.dirname(logfile), exist_ok=True)
    
    with open(logfile, '+w') as f:
        f.write('{} | {}: {}'.format(tile_id, process, error_msg))
    
    return

def has_error(tile_id):
    errors = glob.glob(os.path.join(interim, 'failed', '*.txt'))
    tiles_with_errors = [os.path.basename(error).split('.')[0] for error in errors]
    if tile_id in tiles_with_errors:
        return True
    else:
        return False

In [ ]:
# push our working directories and wrapper classes to the workers on the cluster as well
c.scatter([interim, processed, layers, las, fus, target_epsg, num_cores, has_error, log_error], broadcast=True);

## 1. Classify points in the lidar point cloud

If the original tiles delivered by the vendor included overlapping edges, our retiling may result in duplicated points in the new tiles from overlapping edges of vendor-provided input tiles. In the next step, we will ensure that only one point with unique (X,Y,Z) coordinates are retained in the point cloud data.

In [ ]:
@dask.delayed
def remove_duplicate_points(tile_id):
    if not os.path.exists(os.path.join(interim, 'layers', tile_id + '.lay')):
        try:
            proc = las.lasduplicate(i=os.path.join(interim, 'retiled', tile_id + '.laz'),
                                    unique_xyz=True,
                                    olay=True,
                                    olaydir=layers,
                                    odir=layers)
        except PipelineError as e:
            log_error(tile_id, 'remove_duplicate_points', e.message)
    else: # output file already exists
        pass
    
    return tile_id

In [ ]:
@dask.delayed
def height_above_ground(tile_id):
    if not has_error(tile_id):
        try:
            proc = las.lasheight(i=os.path.join(interim, 'retiled', tile_id + '.laz'),
                                 ilaydir=layers,
                                 olay=True,
                                 classify_below=(-1.0, 7), # noise
                                 olaydir=layers)
        except PipelineError as e:
            log_error(tile_id, 'height_above_ground', e)
    
    else: # this tile already has an error recorded in the error_log
        pass
    
    return tile_id

Now we'll remove points that are isolated as likely noise.

In [ ]:
@dask.delayed
def denoise(tile_id):    
    if not has_error(tile_id):
        try:
            proc = las.lasnoise(i=os.path.join(interim, 'retiled', tile_id + '.laz'),
                                ilaydir=layers,
                                olay=True,
                                step=2, # step-size for each cell in 3x3x3 search area is 2m
                                isolated=10, # points with fewer than 10 neighbors in a 3x3x3 search area
                                olaydir=layers)
        except PipelineError as e:
            log_error(tile_id, 'denoise', e)
    
    else: # this tile already has an error recorded in the error_log
        pass
    
    return tile_id

Now, we'll classify points as building or high vegetation that meet certain criteria for 'planarity' or 'ruggedness'. 

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

In [ ]:
@dask.delayed
def classify(tile_id):
    if not has_error(tile_id):
        try:
            proc = las.lasclassify(i=os.path.join(interim, 'retiled', tile_id + '.laz'),
                                   ilaydir=layers,
                                   step=2.0, # if your data are in meters
                                   planar=0.1, # if your data are in meters
                                   rugged=0.4, # if your data are in meters
                                   olaz=True,
                                   odir=os.path.join(processed, 'points'))
        except PipelineError as e:
            log_error(tile_id, 'classify', e.message)
    
    else: # this tile already has an error recorded in the error_log
        pass
    
    return tile_id

## 2. Generate a bare earth Digital Elevation Model
Generate tiles of the bare earth model. This assumes that there are already ground-classified points

In [ ]:
@dask.delayed
def make_dem(tile_id):
    odir = os.path.join(interim, 'dem_tiles')
    outfile = os.path.join(odir, tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try: 
                proc = las.las2dem(i=os.path.join(processed, 'points', tile_id + '.laz'),
                                   odir=odir,
                                   otif=True, # create tiles as GeoTiff rasters
                                   keep_class=2, # keep ground-classified returns only
                                   step=1, # resolution of output raster, in units of lidar data
                                   thin_with_grid=1, # use a 1 x 1 resolution for creating the TIN for the DEM
                                   extra_pass=True) # uses two passes over data to execute DEM creation more efficiently
            except PipelineError as e:
                log_error(tile_id, 'make_dem', e.message)
    else: # output file already exists
        pass
        
    return tile_id

In [ ]:
@dask.delayed
def add_dem_projection(tile_id):
    basename = tile_id + '.tif'
    infile = os.path.join(interim, 'dem_tiles', basename)
    
    if not has_error(tile_id):
        proc = subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:{}'.format(target_epsg), infile],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        if proc.returncode != 0:
            log_error(tile_id, 'add_dem_projection', proc.stderr.decode())
    
    return tile_id

To create a hillshade layer, we'll first, generate hillshade tiles from the bare earth model.

In [ ]:
@dask.delayed
def make_hillshade(tile_id):
    odir = os.path.join(interim, 'hillshade_tiles')
    outfile = os.path.join(odir, tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try: 
                proc = las.las2dem(i=os.path.join(processed, 'points', tile_id + '.laz'),
                       odir=odir,
                       otif=True, # create tiles as GeoTiffs
                       hillshade=True,
                       keep_class=2, # keep ground-classified returns only
                       step=1, # resolution of output raster, in units of lidar data
                       thin_with_grid=1, # use a 0.5 x 0.5 resolution for creating the TIN for the DEM
                       extra_pass=True) # uses two passes over data to execute DEM creation more efficiently
            
            except PipelineError as e:
                log_error(tile_id, 'make_hillshade', e.message)
    else: # output file already exists
        pass
        
    return tile_id

In [ ]:
@dask.delayed
def add_hillshade_projection(tile_id):
    basename = tile_id + '.tif'
    infile = os.path.join(interim, 'hillshade_tiles', basename)
    
    if not has_error(tile_id):
        proc = subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:{}'.format(target_epsg), infile],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    
        if proc.returncode != 0:
                log_error(tile_id, 'add_hillshade_projection', proc.stderr.decode())
    
    return tile_id

## 3. Identify building footprints
First start by building shapefiles showing building boundaries in each buffered tile.

In [ ]:
@dask.delayed
def make_building_footprint_tiles(tile_id):
    odir = os.path.join(interim, 'building_tiles')
    outfile = os.path.join(odir, tile_id + '.shp')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                proc = las.lasboundary(i=os.path.join(processed, 'points', tile_id + '.laz'),
                                       keep_class=6, # use only building-classified points
                                       disjoint=True, # compute separate polygons for each building
                                       concavity=1, # map concave boundary if edge length >= 1m
                                       oshp=True,
                                       odir=odir)
            except PipelineError as e:
                log_error(tile_id, 'make_building_footprint_tiles', e.message)
    else: # output file already exists
        pass
        
    return tile_id

Generate shapefiles showing the bounding box of each (unbuffered) tile that we'll use to remove buildings that fall in the buffered area.

In [ ]:
@dask.delayed
def unbuffered_tile_boundary(tile_id):
    odir = os.path.join(interim, 'tile_boundaries')
    outfile = os.path.join(odir, tile_id + '.shp')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                proc = las.lasboundary(i=os.path.join(processed, 'points', tile_id + '.laz'),
                                       odir=odir,
                                       oshp=True,
                                       use_tile_bb=True)
            except PipelineError as e:
                log_error(tile_id, 'unbuffered_tile_boundary', e.message)
    else: # output file already exists
        pass
        
    return tile_id

For each shapefile containing polygons of the building boundaries, we'll use the `clean_buffer_polys` function from `pyFIRS.utils` to remove polygons from a tile if their centroid falls in the buffered area of the tile.

In [ ]:
@dask.delayed
def remove_building_buffers(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]

    basename = tile_id + '.shp'
    infile = os.path.join(interim, 'building_tiles', basename)
    tile_shp = os.path.join(interim, 'tile_boundaries', basename)
    odir = os.path.join(processed, 'vectors', 'building_tiles')
    outfile = os.path.join(odir, basename)
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                clean_buffer_polys(infile,
                                   tile_shp,
                                   odir=odir,
                                   simp_tol=3,
                                   simp_topol=True)
            except Exception as e:
                log_error(tile_id, 'remove_building_buffers', e.message)
    else: # output file already exists
        pass
        
    return tile_id

## 4. Create a Canopy Height Model
We're going to switch use a FUSION command line tool to generate a Canopy Height Models (CHMs). 

### Using FUSION's `canopymodel` to generate CHMs
`FUSION` wants to have ground models formatted as .dtm files, for CHM development and for estimating other canopy metrics. Let's generate these ground models first using a 1-meter x-y resolution.

In [ ]:
@dask.delayed
def make_ground_dtm(tile_id):
    infile = os.path.join(processed, 'points', tile_id + '.laz')
    odir = os.path.join(interim, 'dtm_ground_tiles')
    outfile = os.path.join(odir, tile_id + '.dtm')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                proc = fus.gridsurfacecreate(surfacefile=outfile,
                                             cellsize=1,
                                             xyunits='M',
                                             zunits='M',
                                             coordsys=1, # in UTM
                                             zone='10N',
                                             horizdatum=2, # NAD83
                                             vertdatum=2, # NAVD88
                                             datafile=infile,
                                             las_class=2, # keep only ground-classified points
                                             odir=odir) # will make sure output directory is created if doesn't already exist
            except PipelineError as e:
                log_error(tile_id, 'make_ground_dtm', e.message)
    else: # output file already exists
        pass
                
    return tile_id

In [ ]:
@dask.delayed
def make_canopy_model(tile_id):
    infile = os.path.join(processed, 'points', tile_id + '.laz')
    odir = os.path.join(interim, 'chm_tiles')
    outfile = os.path.join(odir, tile_id + '.dtm')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                proc = fus.canopymodel(surfacefile=outfile,
                                       cellsize=0.5, # in meters
                                       xyunits='M',
                                       zunits='M',
                                       coordsys=1, # in UTM
                                       zone='10N', # not in UTM
                                       horizdatum=2, # NAD83
                                       vertdatum=2, # NAVD88
                                       datafiles=infile,
                                       ground=os.path.join(interim, 'dtm_ground_tiles', tile_id + '.dtm'),
                                       median=3, # median smoothing in 3x3 kernel
                                       las_class=(1,2,5), # keep only ground, unclassified, and high veg points
                                       outlier=(-1,110),
                                       asc=True, # also output in ascii format
                                       odir=odir) # will make sure output directory is created if doesn't already exist
                
            except PipelineError as e:
                        log_error(tile_id, 'make_canopy_model', e.message)
    else: # output file already exists
        pass
                
    return tile_id

Convert the ascii files that `canopymodel` generated into GeoTiffs, specifying their projection. Then cleanup the files `canopymodel` generated that we don't care about.

In [ ]:
@dask.delayed
def asc2tif(tile_id):
    infile = os.path.join(interim, 'chm_tiles', tile_id + '.asc')
    outfile = os.path.join(interim, 'chm_tiles', tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                convert_project(infile, '.tif', 'EPSG:{}'.format(target_epsg))
            except Exception as e:
                log_error(tile_id, 'asc2tif', e.message)
    else: # output file already exists
        pass
    
    return tile_id

Clip the canopy height model tiles to remove overlapping areas that were from tile buffering to avoid edge effects.

In [ ]:
@dask.delayed
def remove_canopy_model_buffer(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]
        
    infile = os.path.join(interim, 'chm_tiles', tile_id + '.tif')
    in_shp = os.path.join(interim, 'tile_boundaries', tile_id + '.shp')
    odir = os.path.join(processed, 'rasters', 'chm_tiles')
    outfile = os.path.join(odir, tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                clip_tile_from_shp(infile, 
                                   in_shp, 
                                   odir,
                                   buffer=10)
            
            except Exception as e:
                log_error(tile_id, 'remove_canopy_model_buffer', e.message)
    else: # output file already exists
        pass
    
    return tile_id

In [ ]:
@dask.delayed
def remove_dem_buffer(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]
        
    infile = os.path.join(interim, 'dem_tiles', tile_id + '.tif')
    in_shp = os.path.join(interim, 'tile_boundaries', tile_id + '.shp')
    odir = os.path.join(processed, 'rasters', 'dem_tiles')
    outfile = os.path.join(odir, tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                clip_tile_from_shp(infile, 
                                   in_shp, 
                                   odir,
                                   buffer=10)
            
            except Exception as e:
                log_error(tile_id, 'remove_dem_buffer', e.message)
    else: # output file already exists
        pass
    
    return tile_id

In [ ]:
@dask.delayed
def remove_hillshade_buffer(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]
        
    infile = os.path.join(interim, 'hillshade_tiles', tile_id + '.tif')
    in_shp = os.path.join(interim, 'tile_boundaries', tile_id + '.shp')
    odir = os.path.join(processed, 'rasters', 'hillshade_tiles')
    outfile = os.path.join(odir, tile_id + '.tif')
    
    if not os.path.exists(outfile):
        if not has_error(tile_id):
            try:
                clip_tile_from_shp(infile, 
                                   in_shp, 
                                   odir,
                                   buffer=10)
            
            except Exception as e:
                log_error(tile_id, 'remove_hillshade_buffer', e.message)
    else: # output file already exists
        pass
    
    return tile_id

In [ ]:
@dask.delayed
def tile_done(tile_id, *args, **kwargs):
    if type(tile_id) == list:
        tile_id = tile_id[0]
        
    outfile = os.path.join(interim, 'finished', tile_id + '.txt')
    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    
    with open(outfile, '+a') as f:
        f.write('{}'.format(tile_id))
    
    return tile_id

@dask.delayed
def tiles_done(*args, **kwargs):
    return

## Hand-build the computational graph
Define the recipe for computations.

In [ ]:
tile_ids = set(fname(tile) for tile in
                         glob.glob(os.path.join(interim, 'retiled', '*.laz'))
                        )

failed_tiles = set(fname(tile) for tile in 
                   glob.glob(os.path.join(interim, 'failed', '*.txt'))
                  )

finished_tiles = set(fname(tile) for tile in 
                   glob.glob(os.path.join(interim, 'finished', '*.txt'))
                  )

tiles_to_process = list(tile_ids - failed_tiles - finished_tiles)

print('Found {:,d} tiles to process'.format(len(tiles_to_process)))

In [ ]:
dsk = {}
for tile in tile_ids:
    # pre-processing of lidar point clouds
    dsk['deduped-{}'.format(tile)]=(remove_duplicate_points, tile)
    dsk['normalized-{}'.format(tile)]=(height_above_ground, 'deduped-{}'.format(tile))
    dsk['denoised-{}'.format(tile)]=(denoise, 'normalized-{}'.format(tile)) 
    dsk['redenoised-{}'.format(tile)]=(denoise, 'denoised-{}'.format(tile))
    dsk['classified-{}'.format(tile)]=(classify, 'redenoised-{}'.format(tile))
    dsk['bbox-{}'.format(tile)]=(unbuffered_tile_boundary, 'classified-{}'.format(tile))
    
    # make derivative vector and raster products
    dsk['dem-{}'.format(tile)]=(make_dem, 'classified-{}'.format(tile))
    dsk['ground_dtm-{}'.format(tile)]=(make_ground_dtm, 'classified-{}'.format(tile))
    dsk['canopy-{}'.format(tile)]=(make_canopy_model, 'ground_dtm-{}'.format(tile))
    dsk['hillshade-{}'.format(tile)]=(make_hillshade, 'classified-{}'.format(tile))
    dsk['buildings_buffered-{}'.format(tile)]=(make_building_footprint_tiles, 'classified-{}'.format(tile))
    
    # post-processing of derivative products
    dsk['projected_dem-{}'.format(tile)]=(add_dem_projection, 'dem-{}'.format(tile))
    dsk['projected_hillshade-{}'.format(tile)]=(add_hillshade_projection, 'hillshade-{}'.format(tile))
    dsk['buildings_unbuffered-{}'.format(tile)]=(remove_building_buffers, 
                                                 ['buildings_buffered-{}'.format(tile), 'bbox-{}'.format(tile)])
    dsk['canopy_tif-{}'.format(tile)] = (asc2tif, 'canopy-{}'.format(tile))
    dsk['canopy_clipped-{}'.format(tile)]=(remove_canopy_model_buffer, ['canopy_tif-{}'.format(tile), 'bbox-{}'.format(tile)])
    dsk['dem_clipped-{}'.format(tile)]=(remove_dem_buffer, ['projected_dem-{}'.format(tile), 'bbox-{}'.format(tile)])
    dsk['hillshade_clipped-{}'.format(tile)]=(remove_hillshade_buffer, ['projected_hillshade-{}'.format(tile), 'bbox-{}'.format(tile)])
    
    # a catch-all when a tile is finished processing
    dsk['tile_done-{}'.format(tile)]=(tile_done, ['unbuffered-{}'.format(tile),
                                                  'dem_clipped-{}'.format(tile),
                                                  'hillshade_clipped-{}'.format(tile),
                                                  'buildings_unbuffered-{}'.format(tile),
                                                  'canopy_clipped-{}'.format(tile)])
    
dsk['tiles_done'] = (tiles_done, ['tile_done-{}'.format(tile) for tile in tile_ids])

The computational pipeline, including dependencies of each step in the pipeline, can be specified using a dictionary. 

In [ ]:
example_tile_graph = c.get(dsk, 'tile_done-{}'.format(tiles_to_process[0]))
example_tile_graph.visualize()

Get Dask to determine how to get to the last state of the tile-processing pipeline, building a computational graph.

In [ ]:
tiles_graph = c.get(dsk, 'tiles_done')

In [ ]:
tiles_results = c.compute(tiles_graph) # this might take a while...

In [ ]:
progress(tiles_results)

In [ ]:
failed = glob.glob(os.path.join(interim, 'failed', '*.txt'))
finished = [os.path.basename(tile).split('.')[0] for tile in glob.glob(os.path.join(interim, 'finished', '*.txt'))]

summary = '''
Processing Summary
-------------------
{:>5,d} tiles in acquisition
{:>5,d} tiles previously finished in acquisition

{:>5,d} tiles being processed in this run
{:>5,d} tiles from this run finished

{:>5,d} tiles failed
'''.format(len(tile_ids), 
           len(finished_tiles),
           len(tiles_to_process), 
           len(finished) - (len(tile_ids) - len(tiles_to_process)), 
           len(failed))

total_percent_unfinished = int(70 * (1-len(finished)/len(tile_ids)))
total_percent_finished = int(70 * len(finished)/len(tile_ids))
total_percent_failed = int(70 * len(failed)/len(tile_ids))

this_run_unfinished = int(70 - 70*(len(finished) - (len(tile_ids) - len(tiles_to_process))) / len(tiles_to_process))
this_run_finished = int(70*(len(finished) - (len(tile_ids) - len(tiles_to_process))) / len(tiles_to_process))

print(summary)
print('|' + '=' * this_run_finished + 
      ' '* this_run_unfinished + 
      '!' * total_percent_failed + 
      '|  {:.1%} this run'.format(
          (len(finished) - (len(tile_ids) - len(tiles_to_process))) / len(tiles_to_process)))
print('|' + '=' * total_percent_finished + 
      ' ' * total_percent_unfinished + 
      '!' * total_percent_failed + 
      '|  {:.1%} total'.format(
          len(finished)/len(tile_ids)))

In [ ]:
for filename in failed:
    with open(filename) as f:
        print([line for line in f.readlines() if line.rstrip() != ''])
    print('----------------------')

In [ ]:
# c.cancel(tiles_results)

In [ ]:
# c.close()
# cluster.close()